In [1]:
# Imports

from modules import adquire, cleaning, create_df_checking, outputs


def main(): 
    df_students, df_katas = adquire()    
    df_students, df_katas = cleaning(df_students, df_katas)
    df,hora= create_df_checking(df_students, df_katas)    
    outputs(df,False)


if __name__ == '__main__':
    main()

mcanones presentará la kata counting-sheep-dot-dot-dot


slug,counting-sheep-dot-dot-dot
username,
WHYTEWYLL,True
BSanau,True
senso_ann,False
CarolinaCastioni,False
mariamoyano,True
yuvtorres,True
dfest,True
mcanones,True
Florinda,True
